In [157]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import urllib.parse
import requests
%matplotlib inline
from lxml import html


from io import BytesIO
import requests


req = requests.get('https://yadi.sk/d/iSVP1sZ7AdW7TQ')
data = req.content

df = pd.read_csv(BytesIO(data))

In [159]:
#df

In [160]:
assessments = pd.read_csv('assessments.csv', parse_dates=['date'])

code_module — идентификационный код предмета.

code_presentation — семестр (Идентификационный код).

id_assessment — тест (Идентификационный номер ассессмента).

assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).

date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).

weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

In [161]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0


In [162]:
courses = pd.read_csv('courses.csv')

code_module — предмет (идентификационный код).

code_presentation — семестр (идентификационный код).

module_presentation_length — продолжительность семестра в днях.

In [163]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [164]:
st_assessments = pd.read_csv('studentAssessment.csv')

id_assessment — тест (идентификационный номер).

id_student — идентификационный номер студента.

date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.

is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).

score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

In [165]:
st_assessments.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [166]:
regist = pd.read_csv('studentRegistration.csv')

code_module — предмет (идентификационный код).

code_presentation — семестр (идентификационный код)

id_student — идентификационный номер студента.

date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [167]:
regist.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)

In [168]:
asc_1 = st_assessments.merge(assessments, how='inner', on='id_assessment') 

In [169]:
asc_1

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19,10.0
...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229,0.0


In [170]:
add = asc_1.query('assessment_type=="Exam" and score>40.0')

In [171]:
add

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
52924,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
52925,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
52926,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
52927,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0
52928,24290,560494,230,0,92.0,CCC,2014B,Exam,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...
95978,25368,652617,243,0,56.0,DDD,2014J,Exam,NaN,100.0
95980,25368,652680,243,0,91.0,DDD,2014J,Exam,NaN,100.0
95981,25368,652732,249,0,47.0,DDD,2014J,Exam,NaN,100.0
95982,25368,652965,242,0,87.0,DDD,2014J,Exam,NaN,100.0


In [172]:
add['id_student'].unique #итого 4282 студента


<bound method Series.unique of 52924    559706
52925    559770
52926    560114
52927    560311
52928    560494
          ...  
95978    652617
95980    652680
95981    652732
95982    652965
95984    653319
Name: id_student, Length: 4282, dtype: int64>

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*.
*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [173]:
assessments_student = assessments.merge(st_assessments, how = 'inner', on = 'id_assessment')

In [174]:
# Определение общего кол-ва попыток сдачи тестов
full_test = assessments_student.groupby(['code_module', 'assessment_type', 'id_assessment'], as_index = False) \
            .agg({'code_presentation':'count'}) \
            .rename(columns = {'code_presentation':'total_count'})
full_test

,code_module,assessment_type,id_assessment,total_count
0,AAA,TMA,1752,359
1,AAA,TMA,1753,342
2,AAA,TMA,1754,331
3,AAA,TMA,1755,303
4,AAA,TMA,1756,298
...,...,...,...,...
183,GGG,TMA,37426,612
184,GGG,TMA,37427,549
185,GGG,TMA,37435,586
186,GGG,TMA,37436,540


In [175]:
# Успешные попытки сдачи тестов
success_note = assessments_student.query('score >= 40').groupby(['code_module', 'assessment_type', 'id_assessment'], as_index = False) \
            .agg({'code_presentation':'count'}) \
            .rename(columns = {'code_presentation':'successful_quantity'})

In [176]:
success_note

,code_module,assessment_type,id_assessment,successful_quantity
0,AAA,TMA,1752,352
1,AAA,TMA,1753,330
2,AAA,TMA,1754,325
3,AAA,TMA,1755,294
4,AAA,TMA,1756,290
...,...,...,...,...
183,GGG,TMA,37426,599
184,GGG,TMA,37427,527
185,GGG,TMA,37435,575
186,GGG,TMA,37436,528


In [177]:
success_note['successful_quantity'] / full_test['total_count']

0      0.980501
1      0.964912
2      0.981873
3      0.970297
4      0.973154
         ...   
183    0.978758
184    0.959927
185    0.981229
186    0.977778
187    0.944000
Length: 188, dtype: float64

In [178]:
# Объединение данных и расчет уровня завершаемости
total_test = full_test.merge(success_note, how = 'inner', on = ['code_module', 'assessment_type', 'id_assessment'])
total_test['note_level'] = round(success_note['successful_quantity'] / full_test['total_count'] *100, 2)
total_test.sort_values('note_level')

,code_module,assessment_type,id_assessment,total_count,successful_quantity,note_level
43,BBB,TMA,15020,1776,1040,58.56
94,DDD,TMA,25360,450,334,74.22
82,DDD,TMA,25339,503,392,77.93
88,DDD,TMA,25353,845,664,78.58
48,CCC,CMA,24286,1346,1089,80.91
...,...,...,...,...,...,...
137,FFF,CMA,34907,1234,1234,100.00
121,FFF,CMA,34879,1352,1352,100.00
132,FFF,CMA,34896,747,747,100.00
128,FFF,CMA,34892,857,857,100.00


In [179]:
max_level = total_test.nlargest(1, 'note_level', keep='all')['id_assessment'].to_list()
min_level = total_test.nsmallest(1, 'note_level', keep='all')['id_assessment'].to_list()

In [180]:
print(max_level)
print(min_level)

[34879, 34892, 34896, 34907, 37429]
[15020]


3. По каждому предмету определить средний срок сдачи экзаменов (под сдачей понимается последнее успешное прохождение экзамена студентом)

In [181]:
mean_days = assessments_student.query('assessment_type == "Exam" and score >= 40 and is_banked == 0') \
                        .groupby(['id_student', 'code_module'], as_index = False) \
                        .agg({'date_submitted':'max'}) \
                        .groupby('code_module', as_index = False) \
                        .agg({'date_submitted':'mean'})
mean_days

,code_module,date_submitted
0,CCC,239.345217
1,DDD,237.978566


In [182]:
round(mean_days['date_submitted'],0)

0    239.0
1    238.0
Name: date_submitted, dtype: float64

4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (

In [183]:
popular = regist.groupby(['code_module'], as_index = False) \
                        .agg({'id_student':'nunique'}) \
                        .rename(columns={'id_student':'count_student'}) \
                        .nlargest(3, 'count_student', keep = 'all')
popular

,code_module,count_student
1,BBB,7692
5,FFF,7397
3,DDD,5848


In [184]:
unpopular = regist.query('date_unregistration != "NaN"') \
                        .groupby(['code_module'], as_index = False) \
                        .agg({'id_student':'nunique'}) \
                        .rename(columns={'id_student':'count_student'}) \
                        .nlargest(3, 'count_student', keep = 'all')
unpopular

,code_module,count_student
1,BBB,2314
5,FFF,2249
3,DDD,2065


5. Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. 

In [185]:
assessments_student

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19,10.0,11391,18,0,78.0
1,AAA,2013J,1752,TMA,19,10.0,28400,22,0,70.0
2,AAA,2013J,1752,TMA,19,10.0,31604,17,0,72.0
3,AAA,2013J,1752,TMA,19,10.0,32885,26,0,69.0
4,AAA,2013J,1752,TMA,19,10.0,38053,19,0,79.0
...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37437,TMA,173,0.0,652462,172,0,60.0
173908,GGG,2014J,37437,TMA,173,0.0,652539,176,0,75.0
173909,GGG,2014J,37437,TMA,173,0.0,653157,187,0,70.0
173910,GGG,2014J,37437,TMA,173,0.0,653252,171,0,70.0


In [186]:
total_count = assessments_student.query('assessment_type == "Exam" and is_banked == 0') \
                        .groupby(['code_presentation'], as_index=False) \
                        .agg({'id_student':'count'}) \
                        .rename(columns={'id_student':'total_count'})

In [187]:
total_count

,code_presentation,total_count
0,2013B,602
1,2013J,968
2,2014B,1271
3,2014J,2118


In [188]:
suc_count = assessments_student.query('is_banked == 0 and assessment_type == "Exam" and score>=40') \
                        .groupby(['code_presentation'], as_index=False) \
                        .agg({'id_student':'count'}) \
                        .rename(columns={'id_student':'success_count'})

In [189]:
suc_count

,code_presentation,success_count
0,2013B,504
1,2013J,878
2,2014B,1149
3,2014J,1861


In [190]:
final = total_count.merge(suc_count, how = 'inner', on = 'code_presentation')
final['level'] = round(final.success_count / final.total_count *100, 0)
final.sort_values('level')

,code_presentation,total_count,success_count,level
0,2013B,602,504,84.0
3,2014J,2118,1861,88.0
2,2014B,1271,1149,90.0
1,2013J,968,878,91.0


Самое долгое завершение в 2013B

In [191]:
long = assessments_student.query('is_banked == 0 and assessment_type == "Exam" and score>=40') \
                        .groupby('code_presentation', as_index=False) \
                        .agg({'date_submitted':'mean'})
long.sort_values('date_submitted', ascending = False)

,code_presentation,date_submitted
3,2014J,243.680279
1,2013J,239.509112
2,2014B,232.997389
0,2013B,230.164683


Семестр с самыми долгими средними сроками сдачи курсов - 2014J

6. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен.

In [192]:
# Расчет monetary
monetary = assessments_student.query('assessment_type == "Exam" and is_banked == 0') \
                                    .groupby('id_student', as_index = False) \
                                    .agg({'score':'mean'}) \
                                    .rename(columns = {'score':'Monetary'})

In [193]:
# Расчет recency
recency = assessments_student_data.query('assessment_type == "Exam" and is_banked == 0 and score >= 40') \
                        .groupby('id_student', as_index = False) \
                        .agg({'date_submitted':'mean'}) \
                        .rename(columns={'date_submitted' : 'Recency'})

In [194]:
# Расчет frequency
frequency = assessments_student_data.query('assessment_type == "Exam" and is_banked == 0 and score >= 40') \
                        .groupby('id_student', as_index = False) \
                        .agg({'id_assessment':'count'}) \
                        .rename(columns = {'id_assessment':'count_success'}) \
.merge(assessments_student_data.query('assessment_type == "Exam" and is_banked == 0') \
                        .groupby('id_student', as_index = False) \
                        .agg({'id_assessment':'count'}) \
                        .rename(columns = {'id_assessment':'count_total'}),
     how = 'inner', on ='id_student') 
frequency = frequency.assign( Frequency = frequency.count_success/frequency.count_total*100)

In [195]:
RFM = recency.merge(frequency, how = 'inner', on = 'id_student') \
                .merge(monetary, how = 'inner', on = 'id_student') \
                [['id_student', 'Recency', 'Frequency', 'Monetary']]
RFM.head()

,id_student,Recency,Frequency,Monetary
0,23698,243.0,100.0,80.0
1,24213,236.0,100.0,58.0
2,27116,243.0,100.0,96.0
3,28046,237.0,100.0,40.0
4,28787,243.0,100.0,44.0


In [196]:
level=[0.2, 0.4, 0.6, 0.8]
quantiles = RFM[['Recency', 'Frequency', 'Monetary']].quantile(level).to_dict()
quantiles

{'Recency': {0.2: 231.0, 0.4: 237.0, 0.6: 242.0, 0.8: 243.5},
 'Frequency': {0.2: 100.0, 0.4: 100.0, 0.6: 100.0, 0.8: 100.0},
 'Monetary': {0.2: 53.0, 0.4: 64.0, 0.6: 74.29999999999995, 0.8: 87.0}}

In [197]:
# Присвоение рангов
#Frequency - см выше, квантили неудобны
intervals_F=[0, 50, 100]
RFM['F']=pd.cut(RFM.Frequency, intervals_F, labels=['3', '5'])

In [198]:
# Recency (чем меньше - тем лучше)

intervals_R=[0]
for i in range(len(quantiles['Recency'])): 
    intervals_R.append(quantiles['Recency'][level[i]])
intervals_R.append(RFM.Recency.max())
RFM['R']=pd.cut(RFM.Recency, intervals_R, labels=['5', '4', '3', '2', '1'])

In [199]:
#Monetary чем больше - тем лучше
intervals_M=[0]
for i in range(len(quantiles['Monetary'])): 
    intervals_M.append(quantiles['Monetary'][level[i]])
intervals_M.append(RFM.Monetary.max())
RFM['M']=pd.cut(RFM.Monetary, intervals_M, labels=['1','2','3','4','5'])

In [200]:
RFM=RFM.astype({'R':'string', 'F':'string', 'M':'string'})
RFM['RFM_score'] = RFM['R'] + RFM['F'] + RFM['M']
RFM.head()

,id_student,Recency,Frequency,Monetary,F,R,M,RFM_score
0,23698,243.0,100.0,80.0,5,2,4,254
1,24213,236.0,100.0,58.0,5,4,2,452
2,27116,243.0,100.0,96.0,5,2,5,255
3,28046,237.0,100.0,40.0,5,4,1,451
4,28787,243.0,100.0,44.0,5,2,1,251


In [201]:
# Вводим сегментацию

# Создадим столбец с рангами, чтобы затем заменить его на названия сегментов
RFM['Segment'] = RFM['R'].map(str) + RFM['F'].map(str) + RFM['M'].map(str)

#Описание сегментации. 

segment_map = {
    r'[1-3][3-5][1-3]': 'не справляются-скоро отчислятся', 
    r'[1-3][3-5][4-5]': 'отстающие', 
    r'[3-4][3-5][1-3]': 'отстают, но стараются', 
    r'[3-4][3-5][4-5]': 'хорошисты',  
    r'[4-5][3-5][2-3]': '4+', 
    r'55[4-5]': 'отличники',
    r'5[3-5]1': 'часто с пересдачей',} 

# заменим ранги на названия сегментов
RFM['Segment'] = RFM['Segment'].replace(segment_map, regex=True)
RFM.head(10)

,id_student,Recency,Frequency,Monetary,F,R,M,RFM_score,Segment
0,23698,243.0,100.0,80.0,5,2,4,254,отстающие
1,24213,236.0,100.0,58.0,5,4,2,452,"отстают, но стараются"
2,27116,243.0,100.0,96.0,5,2,5,255,отстающие
3,28046,237.0,100.0,40.0,5,4,1,451,"отстают, но стараются"
4,28787,243.0,100.0,44.0,5,2,1,251,не справляются-скоро отчислятся
5,29411,237.0,100.0,62.0,5,4,2,452,"отстают, но стараются"
6,29420,244.0,100.0,62.0,5,1,2,152,не справляются-скоро отчислятся
7,29639,248.0,100.0,74.0,5,1,3,153,не справляются-скоро отчислятся
8,29764,230.0,100.0,94.0,5,5,5,555,отличники
9,29820,230.0,100.0,76.0,5,5,4,554,отличники


In [202]:
segmentation= RFM.groupby('Segment', as_index=False)  \
                .agg({'id_student':'count'}) \
                .rename(columns={'id_student':'count_student'}) \
                .sort_values('count_student', ascending=False)
segmentation

,Segment,count_student
1,не справляются-скоро отчислятся,1292
4,отстающие,937
3,"отстают, но стараются",625
5,хорошисты,417
0,4+,326
2,отличники,285
6,часто с пересдачей,215


In [210]:
segmentation.columns

Index(['Segment', 'count_student'], dtype='object')

In [1]:
#import matplotlib.pyplot as plt
#plt.figure(figsize=(10, 5))
#sns.barplot(x='Segment', y='count_student', data=segmentation)
